# Example Notebook

Jupyter notebooks are a great tool for data analysis and prototyping. Each engineer will have their own folder under `notebooks/{username}/` to store Jupyter notebooks.

This notebook shows some examples.

Every notebook will start with this code cell. It moves up two directories from where the notebook is, so that we can execute code from the root directory, `transithealth/`. This allows us to import code from across project as well as read/write files using paths relative to the root directory.

In [1]:
import os
os.chdir("../../")

## Importing Code and Accessing Files

Now we can import code from the backend API and run it!

In [2]:
from api.metrics.rideshare import RideshareMetrics
from api.utils.testing import create_test_db

We can also refer to scripts from the offline pipeline using their relative paths from the root.

In [3]:
con, cur = create_test_db(
    scripts=[
        "./pipeline/load/rideshare.sql"
    ],
    tables={
        "rideshare": [
            { "n_trips": 7 },
            { "n_trips": 14 },
            { "n_trips": 3 }
        ]
    }
)

metric = RideshareMetrics(con)
actual = metric.get_max_trips()

expected = { "max_trips": 14 }

assert actual == expected

print(f"Actual:   {actual}")
print(f"Expected: {expected}")

Actual:   {'max_trips': 14}
Expected: {'max_trips': 14}


## Writing SQL

There are two ways to write SQL in a Jupyter notebooks:

1. With the `sqlite3` module built into Python.
2. With the SQL extension for Jupyter notebooks.

Here is an example using Python. The default fetched response does not include the columns, so we have a method in `api.utils.database` to help. We can also use Pandas to load the result into a DataFrame.

In [4]:
import sqlite3

con = sqlite3.connect("./pipeline/database.db")
cur = con.cursor()

rows = cur.execute("""
SELECT *
FROM rideshare
LIMIT 5
""").fetchall()

rows

[('2018-11-01', 1, 1, 500, '2018-44'),
 ('2018-11-01', 50, 43, 19, '2018-44'),
 ('2018-11-01', 50, 44, 17, '2018-44'),
 ('2018-11-01', 50, 46, 4, '2018-44'),
 ('2018-11-01', 50, 47, 6, '2018-44')]

In [5]:
import pandas as pd
from api.utils.database import rows_to_dicts

pd.DataFrame(rows_to_dicts(cur, rows))

,dropoff_community_area,n_trips,pickup_community_area,week,ymd
0,1,500,1,2018-44,2018-11-01
1,43,19,50,2018-44,2018-11-01
2,44,17,50,2018-44,2018-11-01
3,46,4,50,2018-44,2018-11-01
4,47,6,50,2018-44,2018-11-01


Jupyter also supports notebook extensions. This extension allows us to declare a cell as a SQL cell with `%%sql` on the first line of the cell, write queries directly in the cell body, and then view the result as a table.

In [6]:
%%capture
%load_ext sql
%sql sqlite:///pipeline/database.db

In [7]:
%%sql
SELECT *
FROM rideshare
LIMIT 5

 * sqlite:///pipeline/database.db
Done.


ymd,pickup_community_area,dropoff_community_area,n_trips,week
2018-11-01,1,1,500,2018-44
2018-11-01,50,43,19,2018-44
2018-11-01,50,44,17,2018-44
2018-11-01,50,46,4,2018-44
2018-11-01,50,47,6,2018-44


## Querying a Socrata Data Portal

Socrata SQL (SoQL) is a special dialect of SQL that we can use to access datasets from the City of Chicago data portal, as well as other data portals hosted on Socrata.

Below is an example code cell, which uses the `request` module to send a query and get the response as well as Pandas to display the result.

It can take a while to get a response, because you are sending a request to a remote server that will run your SoQL query against the entire dataset. That is why we write scripts in our offline pipeline to aggregate and download data before applying transformations locally.

In [30]:
import pandas as pd
import requests

dataset_json_url = "https://data.cityofchicago.org/resource/wrvz-psew.json"
query = """
SELECT date_trunc_ymd(trip_start_timestamp) as ymd, 
    trip_id, 
    taxi_id, 
    trip_miles, 
    trip_seconds/60 as trip_minutes,
    pickup_community_area,
    dropoff_community_area,
    payment_type
    
WHERE trip_end_timestamp <= "2021-05-01"


LIMIT 100
"""
r = requests.get(dataset_json_url, params={"$query": query})
pd.DataFrame(r.json())

,ymd,trip_id,taxi_id,trip_miles,trip_minutes,pickup_community_area,dropoff_community_area,payment_type
0,2017-01-19T00:00:00.000,88c169f7b6d3068f89207127fcfd2025378f598d,e394089d397e0007d9a4154fa7f6936aba7bd6052e892e...,3.6,15.8166666666666667,6,8,Cash
1,2017-01-18T00:00:00.000,6ff0e41fbbb77e38c270ce422f47590fff4c9792,5875da2f8b2c80730590e971a29f16061bac24b92e13a5...,1.1,7.5166666666666667,28,28,Cash
2,2017-01-15T00:00:00.000,147fecfb4018bdacc16a7d4ac6c4c28a9faad68d,34a67a4a44a0c07c24568bbdee112b8e7877b6b49664a7...,2.2,10.0500000000000000,3,4,Cash
3,2017-01-16T00:00:00.000,9477cc4818ad07c446ebc3b61930518901ba3088,07f6fa4e48d169ab189829ad0db1693534a26518982d79...,21.8,65.3666666666666667,56,1,Credit Card
4,2017-01-16T00:00:00.000,c37055689596fd665c9291d532c3e29d9102775b,3fa0e5b3d416e164aba1cf34cf4b89a2a72e62a7b9ac12...,17.8,54.1166666666666667,32,76,Cash
...,...,...,...,...,...,...,...,...
95,2017-01-18T00:00:00.000,0c88b9e82ad3f336101b68702e82998af5b1567f,3e411cc3ddbde7425617bdf806419ecea09ca0d9b72e0f...,7.1,17.0500000000000000,4,28,Cash
96,2017-01-18T00:00:00.000,477022c180f80106f1bab746d87273f0a40fbf67,19746bb16bdf4cfc8816650883cc6b6f8a7401ceb73a61...,2.1,10.9500000000000000,28,8,Cash
97,2017-01-18T00:00:00.000,47db882b5751350bb6598c605067d70b322b83bd,6742af1fa219cb510a0f217f3e4f2457a065503294bc78...,17.5,50.4333333333333333,28,76,Credit Card
98,2017-01-21T00:00:00.000,8a844516955a4aea01fe96e71e494ba7fd365a38,6898e40854937399e0ef25dad63740d21b205934390907...,1.6,11.5500000000000000,8,8,Cash


In [ ]:
data/extracted/taxi_trips.csv:
    python3 extract/from_data_portal.py \
        --json_url="https://data.cityofchicago.org/resource/wrvz-psew.json" \
        --soql_file="extract/taxi_trips.sql" \
        --output_file="data/extracted/taxi_trips.csv"